In [1]:
### We create our log file

In [2]:
## %matplotlib inline
import matplotlib.pyplot as plt
import requests, re, time
from bs4 import BeautifulSoup
import scraping_class
import pprint
import pandas as pd
import numpy as np
import pickle
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile)
session = requests.session()
session.headers['wmd776@alumni.ku.dk'] = 'wmd776@alumni.ku.dk' 
session.headers['Amalie_Tokkesdal'] = 'Amalie Tokkesdal'
session.headers


{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'wmd776@alumni.ku.dk': 'wmd776@alumni.ku.dk', 'Amalie_Tokkesdal': 'Amalie Tokkesdal'}

In [3]:
import requests

In [4]:
url=[]
for n in range (1,5251,250):
    url.append('https://www.imdb.com/search/title/?title_type=feature&num_votes=20000,&sort=moviemeter,desc&count=250&start=' + str(n) + '&ref_=adv_nxt')
   


links = []
import time
for x in url:
    time.sleep(0.5)
    response = requests.get(x)
    if response.ok:
        html = response.text
    else:
        print('error')
    BeautifulSoup(html,'html.parser')
    link_location = html.split('href="')[1:]
    


    for link_loc in link_location:
        link = link_loc.split('"')[0].strip('title_type')      #-> check om det kan ekskluderes bagefter
        if '/title/' in link:
            links.append(link) 


links = ['https://www.imdb.com/'+link for link in links if not 'plotsummar' in link and not 'title_type' in link and not 'ref' in link and not 'plotsummary' in link and not 'vote' in link] # add the domain to each link
links=set(links)
links


{'https://www.imdb.com//title/tt0067756/',
 'https://www.imdb.com//title/tt0088846/',
 'https://www.imdb.com//title/tt1596350/',
 'https://www.imdb.com//title/tt1483013/',
 'https://www.imdb.com//title/tt0269347/',
 'https://www.imdb.com//title/tt0164181/',
 'https://www.imdb.com//title/tt0065063/',
 'https://www.imdb.com//title/tt2832470/',
 'https://www.imdb.com//title/tt0384642/',
 'https://www.imdb.com//title/tt0103855/',
 'https://www.imdb.com//title/tt1972779/',
 'https://www.imdb.com//title/tt1668191/',
 'https://www.imdb.com//title/tt0452608/',
 'https://www.imdb.com//title/tt1778304/',
 'https://www.imdb.com//title/tt3735246/',
 'https://www.imdb.com//title/tt0245686/',
 'https://www.imdb.com//title/tt0490084/',
 'https://www.imdb.com//title/tt5727282/',
 'https://www.imdb.com//title/tt0070510/',
 'https://www.imdb.com//title/tt1179933/',
 'https://www.imdb.com//title/tt0333780/',
 'https://www.imdb.com//title/tt0327850/',
 'https://www.imdb.com//title/tt1655420/',
 'https://w

In [5]:
links=set(links)
len(links)


5271

In [6]:
import pandas as pd
log = pd.read_csv('log.csv',sep=';') # load the ;-separated log 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

EmptyDataError: No columns to parse from file

In [ ]:
dt = log.dt.values[0]

In [ ]:
import datetime
# Convert the time column 't' to datetime 
#log['dt'] = log.t.apply(datetime.datetime.fromtimestamp)
log['dt'] = pd.to_datetime(log.t,unit='s')

In [ ]:
# Plot the time it took to make the call
plt.figure(figsize=(15,4))
plt.plot(log.dt,log.delta_t)
plt.ylabel('Delta t')
plt.xlabel('Scraping progress')
import datetime

In [ ]:
# Plot the response size over time.
plt.figure(figsize=(15,4))
plt.ylabel('Size of response')
plt.xlabel('Scraping progress')
plt.plot(log.dt,log.response_size)

In [ ]:
## Investigate errors
log.success.value_counts() ## Count errors
## Groupby time
# round the time for aggregation
log['dt_minute'] = log.dt.apply(lambda x: x-datetime.timedelta(seconds=x.second,microseconds=x.microsecond))
log['dt_hour'] = log.dt.apply(lambda x: x-datetime.timedelta(minutes=x.minute,seconds=x.second,microseconds=x.microsecond))
log.groupby('dt_hour').success.mean()

### Parsing HTML with BeautifulSoup

When parsing, we took advantage of the pyhton library BeautifulSoup. 

"BeautifulSoup makes the html tree navigable. 
It allows you to:
    * Search for elements by tag name and/or by attribute.
    * Iterate through them, go up, sideways or down the tree.
    * Furthermore it helps you with standard tasks such as extracting raw text from html,
    which would be a very tedious task if you had to hardcode it using `.split` commands and using your own regular expressions will be unstable."

In [ ]:
def find_budget(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Budget:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))
        
def find_gross(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Cumulative Worldwide Gross:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))
        
def find_grossUSA(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Gross USA:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))

def find_rating(soup):
    for i in soup.find(id="title-overview-widget").find_all("div", class_="ratingValue"):
        return float(i.text.strip().split("/")[0])
        
def find_runtime(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Runtime:":
            return float(i.text.strip().split("\n")[1].split("min")[0].strip())
def find_date(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Release Date:":
            return i.text.strip().split(":")[1].split("(")[0].strip()
        
def find_genre(soup):
    for i in soup.find(id="title-overview-widget").find_all("div", class_="subtext"):
        if str(i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(","))[-3]==(')'):
            return i.text.strip().split("|")[1].replace('\n','').replace(' ','').split(",")
        else:
            return i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(",")


### Parsing and cleaning the data 

Here we use our earlier defined functions to make a dictionary of all the data we need from the description page of each movie, ie. title, budget, the cumulative worldwide gross, the rating, the runtime, date and genre.


In [ ]:
def page_extractor(url):
    r, call_id = connector.get(url, "LOL")
    try:
        soup = BeautifulSoup(r.text, "html.parser")
        title = soup.select("#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > h1")[0].text.split("\xa0")[0]
        budget = find_budget(soup)
        cumulative_worldwide_gross=find_gross(soup)
        gross_usa=find_grossUSA(soup)
        rating = find_rating(soup)
        runtime = find_runtime(soup)
        date= find_date(soup)
        genre = find_genre(soup)
    except:
        soup = None
        title = None
        budget = None
        cumulative_worldwide_gross= None
        gross_usa = None
        rating=None
        runtime = None
        date = None
        genre = None
    return {
        "title": title,
        #"year": year,
        "budget": budget,
        "url": url,
        "cumulative_world_gross": cumulative_worldwide_gross,
        "gross_usa": gross_usa,
        "rating": rating,
        "runtime": runtime,
        "date": date,
        "genre": genre
    }


### Create a list and append information from every movie on the list. 
Remark: We have only chosen top 250 so far, as to not make the coding too slow in progress. If you wish to see the whole list, or a minor part of it, simply change the brackets after 'links'.



In [ ]:
res = []
for idx, i in enumerate(links):
    print(idx + 1, "of", len(links))
    res.append(page_extractor(i))


### Making our list into a dataframe, and applying datetime to the releasedate

A dataframe is basically just when a list has both several columns and rows, if there's only one row it's an array.

Remember to comment on the string lengths need to be over 4 characters.


In [ ]:
df = pd.DataFrame(res).dropna()

df = df[df['date'].str.len() > 4]

def preparedatetime(dataframe):
    dataframe['datetime'] = pd.to_datetime(dataframe['date'], format = '%d %B %Y')
    dataframe['year'] = dataframe.datetime.dt.year
    dataframe['month'] = dataframe.datetime.dt.month
    dataframe['day'] = dataframe.datetime.dt.day
    return dataframe

df = preparedatetime(df)
df

df.to_pickle('all_movies.pkl')
# #This command will load dataset from pkl or csv file
#df = pd.read_pickle('all_movies.pkl')
df

In [ ]:
categories = set()
for i in df["genre"].apply(set).tolist():
    for j in i:
        categories.add(j)
#categories= set(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
#                'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
#                'Reality-TV', 'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller'])
df2 = pd.concat([df, pd.DataFrame(columns=categories)], axis=1)


In [ ]:
def df_dummy(df, col):
    return df.apply(lambda x: col in x["genre"], axis=1)

for i in categories:
    df2[i] = df_dummy(df2, i)
    df2[i] = pd.get_dummies(df2[i], drop_first = True)


df2 = df2.set_index(df['title'])
del df2['title']

df2


# Ratings function (main function)
## Will scrape each ranking URL from relevant data.
### Title is gathered by selector while the rankings are gathered by HTML converted into list which then is calculated

In [ ]:
################### sub function making a list and from list created variables in main function ###################
def find_ratings(soup, rate):        
    for i in soup.select("#main > section > div > table:nth-child(14)")[0].find_all('div', class_='bigcell'):
        rate.append(i.text)
# find_ratings(BeautifulSoup(requests.get("https://www.imdb.com//title/tt0266543/ratings").text, "html.parser"))        
# rate


def ratings_extractor(url):
    r_r, call_id = connector.get(url, "ROFLMAO")
    soup_ = BeautifulSoup(requests.get(url).text, "html.parser")
    rate = []
    try:
        title = soup_.select("#main > section > div > div.subpage_title_block > div > div.parent > h3 > a")[0].text
        find_ratings(soup_,rate)
        check = [float(i) for i in rate]
        rate_all = [float(i) for i in rate][0]
        rate_young = float(round(np.array(list(np.array([float(i) for i in rate])[[1, 2, 6, 7, 11, 12]])).sum()/6, 2)) 
        rate_old = float(round(np.array((np.array([float(i) for i in rate])[[3,4,8,9,13,14]])).sum()/6, 2))
        rate_male = [float(i) for i in rate][5]
        rate_female = [float(i) for i in rate][10]
    except:
        title = None
        rate_all = None
        rate_young = None
        rate_old = None
        rate_male = None
        rate_female = None
    return {
        'title': title,
        'rate_all': rate_all,
        'rate_young': rate_young,
        'rate_old': rate_old,
        'rate_male': rate_male,
        'rate_female': rate_female
        }
ratings_extractor('https://www.imdb.com//title/tt0218817/ratings') #fordi den bliver None, skal de fjernes efter

# Gathering all titles and their rankings using main function. 
## Each ranking url has it's own dictionary. Each dictionary is stored as a list

In [ ]:
# rankings for each films are stored in links_r
links_r = [x+'ratings' for x in links]
res_r = []
# for idx, i in enumerate(links_r):
#     print(idx + 1, "of", len(links_r))
#     res_r.append(ratings_extractor(i))

# Translating the list of dictionaries into a DataFrame
## rankings-DataFrame

In [ ]:
# df_r = pd.DataFrame(res_r).dropna() #dropper de film der mangler ratings fra en bestemt gruppe


# df_r.to_pickle('all_ratings.pkl') #husk kun at brug når data skal uploades/gemmes
df_r = pd.read_pickle('all_ratings.pkl').dropna() #husk kun at brug når data skal downloades/hentes
df_r = df_r.set_index(df_r['title'], drop=True)
del df_r['title']



# Merging movie-attribute-DataFrame with rankings-DataFrame, using innerjoin on the movie titles

In [ ]:
df_master = pd.merge(df2, df_r, on='title', how='inner')

df_master = df_master.drop("Fetih 1453", axis=0)
df_master['cumulative_world_gross']=round(df_master['cumulative_world_gross']/1000000,3)
df_master['budget']=df_master['budget']/1000000
df_master['profit']=(df_master['cumulative_world_gross']-df_master['budget'])
df_master['return'] = round(df_master['cumulative_world_gross']/df_master['budget'],2)
df_master['gross_usa']=round(df_master['gross_usa']/1000000,2)
df_master

df_master = df_master.drop(df_master[df_master['return'] > 15].index)
#note that https://www.imdb.com//title/tt1783232/ has a budget of 18 dollars while grossing 35 million dollar.
df_master.loc[df_master['return'].idxmax()]
#It should've been 18 mio according to google.

df_master

# Deskriptiv statistik

In [ ]:
def category_summary(df, category, categories):
    categories = list(categories)
    stats = ["rate_all", "rate_female", "rate_male", "rate_old", "rate_young"]
    df_ = df[stats + categories]
    df_ = df_[df_[category] == 1]
    return df_[stats].mean() 




print(category_summary(df_master, "Fantasy", ["Fantasy"])) #examples
print(category_summary(df_master, "Horror", ["Horror"])) #exmaples

rated = []
for idx, i in enumerate(categories):
    print(idx+1, 'of', len(categories))
    rated.append(category_summary(df_master, i, [i]))
    
# rated 
df_rated=pd.DataFrame(rated, index=categories)
df_rated

#Check at categories er defineret ud fra hvilke genre der rent faktisk er i datasættet

In [ ]:
def genre_stats(df, category, categories):
    categories = list(categories)
    stats = ["cumulative_world_gross", "budget", 'rating']
    df_ = df[stats + categories]
    df_ = df_[df_[category] == 1]
    return df_[stats].mean() 
#husk at inkluder antallet af observationer pr. genre

print(genre_stats(df_master, "Musical", ["Musical"]))


money = []
for idx, i in enumerate(categories):
    money.append(genre_stats(df_master, i, [i]))
    
# rated 
df_money=pd.DataFrame(money, index=categories)
df_money['genres']=categories
df_money['cumulative_world_gross']=round(df_money['cumulative_world_gross'],2)
df_money['budget']=round(df_money['budget'],2)
df_money['profit']=(df_money['cumulative_world_gross']-df_money['budget'])
df_money['return']=round(df_money['cumulative_world_gross']/df_money['budget'],2)
df_money['rating']=round(df_money['rating'],2)

df_money

In [ ]:
df_master.describe()
df_master['return'].describe()

In [ ]:
import matplotlib.style as style
style.use('seaborn-poster')

barplot1 = df_money.sort_values('cumulative_world_gross', ascending=False).plot.bar(x='genres', y='cumulative_world_gross', title='Mean gross revenue by genre', legend=False, color = '#007ACC')
# barplot[0].set_color('red')
barplot1.set_ylabel('Mean total gross (million $)', weight='bold')
barplot1.set_xlabel('Genre', weight='bold')





df_money = df_money.sort_values(by='return')
my_range = list(range(1,len(df_money.index)+1))
fig, ax = plt.subplots(figsize=(15,8))
plt.hlines(y=my_range, xmin=0, xmax=df_money['return'], color='#007acc', alpha=0.2, linewidth=5)
plt.plot(df_money['return'], my_range, "o", markersize=5, color='#007ACC', alpha=0.6)
ax.set_xlabel('Return', fontsize=20, fontweight='black', color = '#333F4B')
ax.set_ylabel('')
ax.tick_params(axis='both', which='major', labelsize=12)
plt.yticks(my_range, df_money.index)
fig.text(0.05, 0.87, 'Genre', fontsize=20, fontweight='black', color = '#333F4B')
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_smart_bounds(True)
ax.spines['bottom'].set_smart_bounds(True)
ax.spines['bottom'].set_position(('axes', -0.04))
ax.spines['left'].set_position(('axes', 0.015))


barplot2 = df_money.sort_values('budget', ascending=False).plot.bar(x='genres', y='budget', title='Mean budget by genre', legend=False, color = '#007ACC')
# barplot[0].set_color('red')
barplot2.set_ylabel('Mean budget (million $)', weight='bold')
barplot2.set_xlabel('Genre', weight='bold')

df_money.sort_values('return', ascending=False).plot.bar(x='genres', y='return')



scatter1 = df_master.plot.scatter(x='budget', y='cumulative_world_gross', title='Budget and total gross correlation', legend=False, color = '#007ACC')
# barplot[0].set_color('red')
scatter1.set_ylabel('Mean total gross (million $)', weight='bold')
scatter1.set_xlabel('budget (million $)', weight='bold')

scatter2 = df_master.plot.scatter(x='rating', y='return', title='Rating and return', legend=False, color = '#007ACC')
# barplot[0].set_color('red')
scatter2.set_ylabel('Return (gross/budget)', weight='bold')
scatter2.set_ylim([0,12])
scatter2.set_xlabel('Rating', weight='bold')




ax = sns.boxplot(y='return', data=df_master).set_title('Boxplots of return (Total gross / budget)')
ax.set_ylabel('Return')

## Machine Learning
Importing the relevant packages

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

### Defining the features and target from our dataframe

In [ ]:
X = df_master[['budget', 'rating', 'runtime', *list(categories)]]
X_r = df_master[['rating', 'runtime', *list(categories)]]
y = df_master['cumulative_world_gross']
y_r = df_master['cumulative_world_gross']/df_master['budget']

from sklearn.model_selection import train_test_split

In [ ]:
def rmse(y_pred, y_true):
    return np.sqrt(mse(y_pred, y_true))

output=[]

for random_state in range(10):
    X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=1/2, random_state=1)
    
    reg = LinearRegression().fit(X_train, y_train)
    reg.predict(X_test)
    
    output.append([rmse(reg.predict(X_val), y_val),
                  rmse(reg.predict(X_test), y_test)])
    
print(pd.DataFrame(output, columns=['test', 'validation']))

print(pd.DataFrame(output, columns=['test', 'validation']).describe())

### Finding the optimal lambda, by OLS

In [ ]:
from sklearn.model_selection import KFold
kfolds = KFold(n_splits=10) #Splits the data 10 times into evenly sized bins. 
folds = list()

lambdas = np.logspace(-4,4,10)
# outer loop: lambdas
rmseCV = []
perform = []

for lambda_ in lambdas:    
    # inner loop: folds. For each of the folds (Splits),
    rmseCV_ = []    
    for train_idx, val_idx in kfolds.split(X_dev, y_dev):        
        # train model and compute MSE on test fold
        pipe_OLSCV = make_pipeline(ColumnTransformer(
                                    [("Continous", make_pipeline(
                                         StandardScaler(),
                                         PolynomialFeatures()
                                     ), [0,2]), 
                                      ("", 'passthrough', [i for i in range(3, X.shape[1])])
                                     ]),
                                    LinearRegression())
        X_train, y_train, = X_dev.iloc[train_idx], y_dev[train_idx] #Selection where we throw in the indices and take out the data
        X_val, y_val = X_dev.iloc[val_idx], y_dev[val_idx] 
        pipe_OLSCV.fit(X_train, y_train)  #Fit the model on the training data, estimation
        rmseCV_.append(rmse(pipe_OLSCV.predict(X_val), y_val))   #Apply the model to the validation data and store it in an empty list
    #y_val_pred = pipe_lassoCV.predict(X_val)
    #y_train_pred = pipe_lassoCV.predict(X_train)
    #y_test_pred = pipe_lassoCV.predict(X_test)
    #perform.append(mse(y_pred, y_val))
    # store result    
    rmseCV.append(sum(rmseCV_)/len(rmseCV_)) 
    
# convert to DataFrame
lambdaCV = pd.DataFrame(rmseCV, index=lambdas)

lambdaCV

optimal_lambda = lambdaCV.mean(axis=1).nsmallest(1)
optimal_lambda

In [ ]:
from sklearn.model_selection import KFold
kfolds = KFold(n_splits=10) #Splits the data 10 times into evenly sized bins. 
folds = list()

rmseCV = []
perform = []

for train_idx, val_idx in kfolds.split(X_dev, y_dev):        
    # train model and compute MSE on test fold
    pipe_OLSCV = make_pipeline(ColumnTransformer(
                                [("Continous", make_pipeline(
                                     StandardScaler(),
                                     PolynomialFeatures()
                                 ), [0,2]), 
                                  ("", 'passthrough', [i for i in range(3, X.shape[1])])
                                 ]),
                                LinearRegression())
    X_train, y_train, = X_dev.iloc[train_idx], y_dev[train_idx] #Selection where we throw in the indices and take out the data
    X_val, y_val = X_dev.iloc[val_idx], y_dev[val_idx] 
    pipe_OLSCV.fit(X_train, y_train)  #Fit the model on the training data, estimation
    rmseCV.append(rmse(pipe_OLSCV.predict(X_val), y_val))   #Apply the model to the validation data and store it in an empty list
#y_val_pred = pipe_lassoCV.predict(X_val)
#y_train_pred = pipe_lassoCV.predict(X_train)
#y_test_pred = pipe_lassoCV.predict(X_test)
#perform.append(mse(y_pred, y_val))
# store result    
rmseCV.append(sum(rmseCV_)/len(rmseCV_)) 
    

In [ ]:
pipe_OLSCV.named_steps['linearregression'].coef_

### Predicting of our target using the three different datasets: Train, validation and test

In [ ]:
y_val_pred = pipe_lassoCV.predict(X_val)
y_train_pred = pipe_lassoCV.predict(X_train)
y_test_pred = pipe_lassoCV.predict(X_test)
    #perform.append(mse(y_pred, y_val))

## Machine learning, OLS with Lasso

From computing the descriptive statistics of RMSE for the train data on  the test and validation data we can conclude we can split the train data and use this to fit the model. This is because the descriptive statistics are quite similar and we therefore presume our model i generalizing enough. (Balanced).

In [ ]:
from sklearn.model_selection import KFold
kfolds = KFold(n_splits=10) #Splits the data 10 times into evenly sized bins. 
folds = list()

lambdas = np.logspace(-2,4,50)
# outer loop: lambdas
rmseCV = []
perform = []

for lambda_ in lambdas:    
    # inner loop: folds. For each of the folds (Splits),
    rmseCV_ = []    
    for train_idx, val_idx in kfolds.split(X_dev, y_dev):        
        # train model and compute MSE on test fold
        pipe_lassoCV = make_pipeline(ColumnTransformer(
                                    [("Continous", make_pipeline(
                                         StandardScaler(),
                                         PolynomialFeatures()
                                     ), [0,2]), 
                                      ("", 'passthrough', [i for i in range(3, X.shape[1])])
                                     ]),
                                    Lasso(alpha=lambda_, random_state=1))
        X_train, y_train, = X_dev.iloc[train_idx], y_dev[train_idx] #Selection where we throw in the indices and take out the data
        X_val, y_val = X_dev.iloc[val_idx], y_dev[val_idx] 
        pipe_lassoCV.fit(X_train, y_train)  #Fit the model on the training data, estimation
        rmseCV_.append(rmse(pipe_lassoCV.predict(X_val), y_val))   #Apply the model to the validation data and store it in an empty list
    #y_val_pred = pipe_lassoCV.predict(X_val)
    #y_train_pred = pipe_lassoCV.predict(X_train)
    #y_test_pred = pipe_lassoCV.predict(X_test)
    #perform.append(mse(y_pred, y_val))
    # store result    
    rmseCV.append(sum(rmseCV_)/len(rmseCV_)) 
    
# convert to DataFrame
lambdaCV = pd.DataFrame(rmseCV, index=lambdas)

lambdaCV

optimal_lambda = lambdaCV.mean(axis=1).nsmallest(1)
optimal_lambda

In [ ]:
pipe_final = make_pipeline(ColumnTransformer(
                            [("Continous", make_pipeline(
                                 StandardScaler(),
                                 PolynomialFeatures()
                             ), [0,2]), 
                              ("", 'passthrough', [i for i in range(3, X.shape[1])])
                             ]),
                            Lasso(alpha=0.359381, random_state=1))



In [ ]:
the_fit = pipe_final.fit(X_dev, y_dev)

the_fit.named_steps['lasso'].coef_

Lasso_coefficients = reg.coef_
Lasso_intercept = reg.intercept_


print(Lasso_coefficients)
print(Lasso_intercept)

#Zip eller plot dem sammen

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = \
    learning_curve(estimator=pipe_lassoCV,
                   X=X_train,
                   y=y_train,
                   train_sizes=np.arange(0.2, 1, .01),
                   scoring='neg_mean_squared_error',                 
                   cv=3)
    
mse_ = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                     'Test':-test_scores.mean(axis=1)})\
        .set_index(pd.Index(train_sizes,name='sample size'))    
print(mse_.head(3))

In [ ]:
f_learn, ax = plt.subplots(figsize=(10,4))
ax.fill_between(train_sizes,
                -test_scores.mean(1) + test_scores.std(1)*1.96,
                -test_scores.mean(1) - test_scores.std(1)*1.96, 
                alpha=0.25, label ='Test', color='blue')

ax.fill_between(train_sizes,
                -train_scores.mean(1) + train_scores.std(1)*1.96,
                -train_scores.mean(1) - train_scores.std(1)*1.96, 
                alpha=0.25, label='Train', color='orange')
ax.set_ylabel('Mean squared error')
ax.set_yscale('log')
ax.legend()

In [ ]:
from sklearn.model_selection import validation_curve

train_scores, test_scores = \
    validation_curve(estimator=pipe_lassoCV,
                     X=X_dev,
                     y=y_dev,
                     param_name='lasso__alpha',
                     param_range=lambdas,
                     scoring='neg_mean_squared_error',                 
                     cv=3)

mse_score = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                          'Validation':-test_scores.mean(axis=1),
                          'lambda':lambdas})\
              .set_index('lambda')   
print(mse_score.Validation.nsmallest(1))

In [ ]:
mse_score.plot(logx=True, logy=True)

#overfitter fordi den er bedre på train end validation

## Prediction of the cumulative gross in relation to the budget

In [ ]:
def rmse(y_pred, y_true):
    return np.sqrt(mse(y_pred, y_true))

output=[]

for random_state in range(10):
    X_r_dev, X_r_test, y_r_dev, y_r_test = train_test_split(X_r, y_r, test_size=1/3, random_state=1)
    X_r_train, X_r_val, y_r_train, y_r_val = train_test_split(X_r_dev, y_r_dev, test_size=1/2, random_state=1)
    
    reg = LinearRegression().fit(X_r_train, y_r_train)
    reg.predict(X_r_test)
    
    output.append([rmse(reg.predict(X_r_val), y_r_val),
                  rmse(reg.predict(X_r_test), y_r_test)])

print(pd.DataFrame(output, columns=['test', 'validation']))

print(pd.DataFrame(output, columns=['test', 'validation']).describe())

In [ ]:
from sklearn.model_selection import KFold
kfolds = KFold(n_splits=10) #Splits the data 10 times into evenly sized bins. 
folds = list()

lambdas = np.logspace(-4,4,10)
# outer loop: lambdas
rmseCV = []
perform = []

for lambda_ in lambdas:    
    # inner loop: folds. For each of the folds (Splits),
    rmseCV_ = []    
    for train_idx, val_idx in kfolds.split(X_r_dev, y_r_dev):        
        # train model and compute MSE on test fold
        pipe_lasso_rCV = make_pipeline(ColumnTransformer(
                                    [("Continous", make_pipeline(
                                         StandardScaler(),
                                         PolynomialFeatures()
                                     ), [0,1]), 
                                      ("", 'passthrough', [i for i in range(2, X_r.shape[1])])
                                     ]),
                                    Lasso(alpha=lambda_, random_state=1))
        X_r_train, y_r_train, = X_r_dev.iloc[train_idx], y_r_dev[train_idx] #Selection where we throw in the indices and take out the data
        X_r_val, y_r_val = X_r_dev.iloc[val_idx], y_r_dev[val_idx] 
        pipe_lasso_rCV.fit(X_r_train, y_r_train)  #Fit the model on the training data, estimation
        rmseCV_.append(rmse(pipe_lasso_rCV.predict(X_r_val), y_r_val))   #Apply the model to the validation data and store it in an empty list
    #y_val_pred = pipe_lassoCV.predict(X_val)
    #y_train_pred = pipe_lassoCV.predict(X_train)
    #y_test_pred = pipe_lassoCV.predict(X_test)
    #perform.append(mse(y_pred, y_val))
    # store result    
    rmseCV.append(sum(rmseCV_)/len(rmseCV_)) 
    
# convert to DataFrame
lambdaCV = pd.DataFrame(rmseCV, index=lambdas)

lambdaCV

optimal_lambda = lambdaCV.mean(axis=1).nsmallest(1)
optimal_lambda

In [ ]:
pipe_optimal_r = make_pipeline(ColumnTransformer(
                            [("Continous", make_pipeline(
                                 StandardScaler(),
                                 PolynomialFeatures()
                             ), [0,2]), 
                              ("", 'passthrough', [i for i in range(3, X.shape[1])])
                             ]),
                            Lasso(alpha=0.005995, random_state=1))

In [ ]:
fit_r = pipe_optimal_r.fit(X_dev, y_dev)

fit_r.named_steps['lasso'].coef_

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = \
    learning_curve(estimator=pipe_lassoCV,
                   X=X_train,
                   y=y_train,
                   train_sizes=np.arange(0.2, 1, .01),
                   scoring='neg_mean_squared_error',         
                   cv=3)
    
mse_ = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                     'Test':-test_scores.mean(axis=1)})\
        .set_index(pd.Index(train_sizes,name='sample size'))    
print(mse_.head(3))

In [ ]:
f_learn, ax = plt.subplots(figsize=(10,4))
ax.fill_between(train_sizes,
                -test_scores.mean(1) + test_scores.std(1)*1.96,
                -test_scores.mean(1) - test_scores.std(1)*1.96, 
                alpha=0.25, label ='Test', color='blue')

ax.fill_between(train_sizes,
                -train_scores.mean(1) + train_scores.std(1)*1.96,
                -train_scores.mean(1) - train_scores.std(1)*1.96, 
                alpha=0.25, label='Train', color='orange')
ax.set_ylabel('Mean squared error')
ax.set_yscale('log')
ax.legend()

In [ ]:
from sklearn.model_selection import validation_curve

train_scores, test_scores = \
    validation_curve(estimator=pipe_lassoCV,
                     X=X_dev,
                     y=y_dev,
                     param_name='lasso__alpha',
                     param_range=lambdas,
                     scoring='neg_mean_squared_error',                 
                     cv=3)

mse_score = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                          'Validation':-test_scores.mean(axis=1),
                          'lambda':lambdas})\
              .set_index('lambda')   
print(mse_score.Validation.nsmallest(1))

In [ ]:
mse_score.plot(logx=True, logy=True)

#overfitter fordi den er bedre på train end validation

# Code for plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns

plt.style.use('ggplot')
%matplotlib inline

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title